In [2]:
from specutils import Spectrum1D
from astropy.nddata import StdDevUncertainty
from astropy.io import fits
from datetime import date
from pandas import to_datetime
import pandas as pd
#from scripts.ingests.utils import * #scripts do not need anything from ingest
import numpy.ma as ma
import matplotlib.pyplot as plt
import os
import astropy.units as u
import astropy  
import seaborn as sns
from astropy.io import fits
from astropy.io import ascii
from glob import glob
import astropy.io.fits as fits
#to edit csv
from csv import writer
from csv import reader
import csv 
# to use vizier tables
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad
from astroquery.xmatch import XMatch
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from astropy.coordinates import SkyCoord # a "sky coordinate" object. Helpful for dealing with locations in space or in the sky

# Astropy's Table tools
from astropy.table import Table, vstack, join

## convert sexagesimal to degrees
import numpy as np
from astropy import units as u
from astropy.coordinates import Angle

#convert dates to MJD
from astropy.time import Time
#slit width from paper
#exposure time form vizier catalog

import numpy.ma as ma
import numpy as np

Could not import regions, which is required for some of the functionalities of this module.


In [4]:
vizier_code = "J/A+A/564/A55/table1" # store the vizier table code into a variable
#may be able to download in degrees

query_catalog = Vizier() # Create a vizier query object

query_catalog.ROW_LIMIT = -1 # Turn off the row limit

manja2014 = query_catalog.get_catalogs(vizier_code)[0] #downloads the table

manja2014
RA_sexa = list(manja2014['RAJ2000'][1:7])
dec_sexa = list(manja2014['DEJ2000'][1:7])

RA_convert = [x +" hours" for x in RA_sexa]
dec_convert = [x +" degrees" for x in dec_sexa] 

ra_degrees = Angle(RA_convert).degree
dec_degrees =Angle(dec_convert).degree

print(dec_convert)

['-44 05 06 degrees', '-77 39 58 degrees', '-61 51 27 degrees', '-81 40 29 degrees', '+29 21 21 degrees', '-21 36 08 degrees']


In [62]:
df = pd.read_csv("Manja14_spectra - Sheet1.csv")
df['RA'] = ra_degrees #want these in string degrees
df['dec'] = dec_degrees
list(df.columns)
df.to_csv('Manja14_spectra_adjusted.csv', index = False)

In [63]:
table_file='Manja14_spectra_adjusted.csv'
data = Table.read(table_file)
original_data_dir = 'ISAAC_Manjavacas2014/' #where the old fits files are
fits_data_dir = 'Manj14/' #where we want the new fits files to go 

#goes through table of data
#this is the part where you input values for each variable
for row in data:
    object = row['Source'] #row is defined so it loops through each row to 
    #logger.debug(f'source: {object}')
    
    spectrum_url = row['Spectrum'] #goes to specific column entry finds the url 

    file = os.path.basename(spectrum_url)
    
    full_name = (original_data_dir + file)
    
    files = glob('ISAAC_Manjavacas2014/*.fits')

    for file_name in files:
        data = fits.getdata(file_name)
        name = file_name[:-len('.fits')] # Remove .fits from the file name
    
    spectrum_table = Table(data)
    
    history1 = f'Original file: {file}' #gives orginal name of file
    history2 = 'This file generated by SIMPLE-db/scripts/ingests/rewrite_spectra.py'  #shows where file came from 
    history = (history1 +', ' + history2) #made into tuple so it is a hashable type
    wavelength_unit = u.micron   #using astropy to defien units 
    flux_unit = u.erg/u.cm/u.cm/u.s/u.Angstrom

    comment1 = 'Manja14 FITS file did not provide flux uncertainty'
    comment = (comment1) #add next comment no tuple bc fits file does not want an iterable item

    wavelength_1 = spectrum_table['col1'] * wavelength_unit   #multiplying everythign by untis to convert

    
    #All the key words should be in one dictionary to pass to the function to write the header
    #REQUIRED KEY WORDSW
    
    #turn these into a dictionary 
    dict = {
        'VOCLASS' : 'Spectrum-1.0',
        'VOPUB' : 'SIMPLE Archive' ,
        'title' : 'ISAAC Spectra of Brown Dwarfs from Manjavacas et. al. 2014' ,
        'RA' : row['RA'] , #from the vizier catalog, put in csv table
        'dec' : row['dec'] , #from the vizier catalog, put in csv table
        'time' : (Time(to_datetime(row['start time'])).jd + Time(to_datetime(row['stop time'])).jd) /2,
        'exposure_time' : row['exposure time'] ,
        'bandpass' : 'JHK' ,#get from paper
        'aperture' :'0.3' ,
    
        #OTHER KEYWORDS
    
        'time_start' : Time(to_datetime(row['start time'])).jd , #turns dates into accepetable format then converts to Time object then to MJD
        'time_stop' : Time(to_datetime(row['stop time'])).jd , #dates r orginally in month day, year
        'bibcode' : '2014A&A...564A..55M' ,
        'instrument' : row['instrument'] ,
        'obs_date' : to_datetime(row['observation_date']) ,
        'author' : 'Manjavacas, E. ; Bonnefoy, M. ; Schlieder, J. E. ; Allard, F. ; Rojo, P. ; Goldman, B. ; Chauvin, G. ; Homeier, D. ; Lodieu, N. ; Henning, T.', 
        'reference_doi' : '10.1051/0004-6361/201323016'  ,
        'telescope' : row['telescope'] ,
        'history' : history ,
        'wavelength' : tuple(spectrum_table['col0'].data) * wavelength_unit , #multiplying everythign by untis to convert
        'flux' : tuple(spectrum_table['col1'].data) * flux_unit, # spectrum_table['col1'] * flux_unit, 
        # flux_unc : 'took out flux uncertainty'  #TOOK OUT FLUX_UNC BC MANJA 14 ONLY HAD 2 COLUMNS
        'comment': comment
        } 
    
    #only 1 columns, does not have flux uncertaitnty 
   

    spectrum_data_out = Table({'wavelength': spectrum_table['col0'] * wavelength_unit , 'flux':spectrum_table['col1'] * flux_unit }) #TOOK OUT FLUX_UNC
    
    hdu1 = fits.BinTableHDU(data = spectrum_data_out) 
    
    hdu1.header['EXTNAME'] = 'SPECTRUM' #prints out different headers
    hdu1.header.set('Spectrum', str(object), 'Object Name')



In [64]:
def header_func(**dict):
    header = fits.Header()
    header.set('EXTNAME', 'PRIMARY', 'name of this extension')
    # IVOA SpectrumDM keywords REQUIRED
    header.set('VOCLASS', dict['VOCLASS'],'VO Data Model') 
    header.set('VOPUB',dict['VOPUB'],'VO Publisher ID URI') # uniform research identifier 
    header.set('TITLE', dict['title'], 'data set title') 
    header.set('OBJECT', object, 'name of observed object')
    header.set('RA', dict['RA'] , '[deg] Pointing position') 
    header.set('DEC', dict['dec'] , '[deg] Pointing position') 
    header.set('TMID', dict['time'], '[d] MJD mid expsoure')
    header.set('TSTART', dict['time_start'], '[d] MJD start time')
    header.set('TSTOP', dict['time_stop'], '[d] MJD stop time')
    header.set('TELAPSE', dict['exposure_time'],'[s] Total elapsed time')
    header.set('SPEC_VAL', dict['bandpass'], '[angstrom] Characteristic spec coord')

    header.set('SPEC_BW', (max(wavelength_1).value - min(wavelength_1.value)), f"[{wavelength_1.unit:FITS}] Width of spectrum")
    header.set('TDMIN1', min(wavelength_1).value, f"[{wavelength_1.unit:FITS}] starting wavelength") 
    header.set('TDMAX1', max(wavelength_1).value, f"[{wavelength_1.unit:FITS}] ending wavelength")
    header.set('APERTURE', dict['aperture'],'[arcsec]slit width') #from paper
    header.set('AUTHOR', dict['author'], 'author of the data') 
    # Other IVOA SpectrumDM keywords
    header.set('VOREF', dict['bibcode'], 'bibcode' )
    header.set('DATE', date.today().strftime("%Y-%m-%d"), 'date of file creation')
    header.set('INSTRUME', dict['instrument'], 'name of instrument')
    header.set('DATE-OBS', dict['obs_date'].strftime("%Y-%m-%d"), 'date of the observation')
    header.set('REFERENC', dict['reference_doi'], 'bibliographic reference')
    header.set('TELESCOP', dict['telescope'], 'name of telescope')
    header.set('HISTORY', dict['history'])
    header.set('COMMENT', dict['comment'])


    return header # function ends here

header_func(**dict)


EXTNAME = 'PRIMARY '           / name of this extension                         
VOCLASS = 'Spectrum-1.0'       / VO Data Model                                  
VOPUB   = 'SIMPLE Archive'     / VO Publisher ID URI                            
TITLE   = 'ISAAC Spectra of Brown Dwarfs from Manjavacas et. al. 2014' / data se
OBJECT  = '2MASS J22134491-2136079' / name of observed object                   
RA      =    333.4370833333332 / [deg] Pointing position                        
DEC     =   -21.60222222222222 / [deg] Pointing position                        
TMID    =            2456510.5 / [d] MJD mid expsoure                           
TSTART  =            2456510.5 / [d] MJD start time                             
TSTOP   =            2456510.5 / [d] MJD stop time                              
TELAPSE =                 1960 / [s] Total elapsed time                         
SPEC_VAL= 'JHK     '           / [angstrom] Characteristic spec coord           
SPEC_BW = 1.98562162462970E-

In [65]:
table_file='Manja14_spectra_adjusted.csv'
data = Table.read(table_file)
original_data_dir = 'ISAAC_Manjavacas2014/' #where the old fits files are
fits_data_dir = 'Manj14/' #where we want the new fits files to go 

for row in data:
    object = row['Source'] #row is defined so it loops through each row to 
    #logger.debug(f'source: {object}')
    
    spectrum_url = row['Spectrum']
    
    
    file = os.path.basename(spectrum_url)
    
    full_name = (original_data_dir + file)
    
    files = glob('ISAAC_Manjavacas2014/*.fits')

    for file_name in files:
        data = fits.getdata(file_name)
        name = file_name[:-len('.fits')]

    hdu0 = fits.PrimaryHDU(header=header_func(**dict)) #not sure what this menas 

    spectrum_mef = fits.HDUList([hdu0,hdu1])#hdu0 is header and hdu1 is data
    
    file_root = os.path.splitext(file)[0] #split the path name into a pair root and ext so the root is just the ext [0] is the name of the file wihtout the .csv
    fits_filename = fits_data_dir + file_root + '.fits' #turns into fits files by putting it in new folder that we defined at begining and adding name of file then .fits
    try:
        spectrum_mef.writeto(fits_filename, overwrite=True)
        # SHOULD BE: spectrum.write(fits_filename, format='tabular-fits', overwrite=True, update_header=True)
        #logger.info(f'Wrote {fits_filename}')
    except:
        raise SimpleError
    print(file)

2M0032_ISAACspec_JHK_nocosmics.fits
ChaJ1305_ISAACspec_JHK_nocosmics_2_dered.fits
2M2322_ISAACspec_JHK_nocosmics.fits
2M2126_ISAACspec_JHK_nocosmics.fits
2M2208_ISAACspec_JHK_nocosmics.fits
2M2213_ISAACspec_JHK_nocosmic.fits


In [23]:
spec1d_fits_rt = fits.open(fits_filename)
header_rt = spec1d_fits_rt[0].header
spec1d_rt = Spectrum1D.read(fits_filename) #reads the fits file